Import useful modules 

In [ ]:
import sys 
import os
### discomment above two line ones to add the feelpp path with nirb module in the python environement  
# feelpath = os.path.join('/data/home/elarif/devel/feelpp/build/python/install/lib/python3/dist-packages')
# sys.path.append(feelpath)
print(sys.path) 
from feelpp.mor.nirb.nirb import *
from feelpp.mor.nirb.utils import init_feelpp_environment
import time
import json 


Choose config file

In [13]:
config_file =os.path.join("/data/home/elarif/devel/feelpp/python/pyfeelpp-mor/feelpp/mor/nirb/model/square/square.cfg")
N = None # number of initiale snapshot 
greedy_tol = 1.e-5 # convergence criteria of Greedy method 
Nmax_greedy = 100 
outdir = None 

Set feelpp environment 

In [ ]:
cfg = feelpp.readCfg(config_file)
toolboxType = cfg['nirb']['toolboxType']
e = init_feelpp_environment(toolboxType, config_file)

nirb_file = feelpp.Environment.expand(cfg['nirb']['filename'])
config_nirb = feelpp.readJson(nirb_file)['nirb']

Get number of initial snapshots. By defaults, it will take the value filled in the model path. 

In [ ]:
nbSnap=N
if nbSnap==None:
    nbSnap = config_nirb['nbSnapshots']

Set the output path 

In [ ]:
doGreedy = config_nirb['greedy-generation']
doRectification = config_nirb['doRectification']
rectPath = ["noRect", "Rect"][doRectification]
greedyPath = ["noGreedy", "Greedy"][doGreedy]

if outdir is None:
    RESPATH = f"results/{rectPath}/{greedyPath}"
else:
    RESPATH = outdir

Initialize nirb offline class 

In [ ]:
nirb_off = nirbOffline(initCoarse=True, **config_nirb)

Generate L2 and H1 operators in fine mesh. If `coarse=True`, It generate those operators in coarse mesh too (this is needed for Greedy method). 

In [16]:
start=time.time()

nirb_off.generateOperators(coarse=True)

Get nirb reduced basis functions 

In [ ]:
Xi_train=None

if doGreedy:
    _, Xi_train, _ = nirb_off.initProblemGreedy(500, greedy_tol, Nmax=Nmax_greedy, Xi_train=Xi_train, computeCoarse=True, samplingMode="log-random")
else:
    Xi_train = nirb_off.initProblem(nbSnap)
nirb_off.generateReducedBasis(regulParam=1.e-10)

print("Is L2 orthonormalized ?", nirb_off.checkL2Orthonormalized())


Save datas. 

If `outdir`is given, the results will be saved on this path. otherwise they are saved in : "HOME/feelppdb/nirb/{ToolboxType}/np_{k}/{RESPATH}" where `k` is the number of process. 

In [ ]:
nirb_off.saveData(RESPATH, force=True)

finish=time.time() 

Get infos 

In [ ]:
info = nirb_off.getInformations()

if feelpp.Environment.isMasterRank():
    print(json.dumps(info, sort_keys=True, indent=4))
    print(f"[NIRB] Offline Elapsed time = ", finish-start)
    print(f"[NIRB] Offline part Done !")